In [60]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
import pickle as pkl
import numpy as np
import time

## Load data

In [64]:
with open("data/X_train224.pkl", "rb") as input_file:
     X_train,y_train = pkl.load(input_file)
X_train = X_train.astype(np.float32)/255

In [65]:
X_train.shape, y_train.shape

((3000, 224, 224, 3), (3000,))

In [66]:
y_train = y_train - 1

In [67]:
y_train_OH = np.zeros((3000,3))
y_train_OH[np.arange(3000), y_train] = 1

In [68]:
num_train,height,weight,channels = X_train.shape
np.random.seed(10)
index = np.random.choice(num_train,num_train,replace=False)
X_train = X_train[index]
y_train_OH = y_train_OH[index]

num_val = int(num_train*0.3)
X_val = X_train[:num_val]
y_val_OH = y_train_OH[:num_val]

X_train = X_train[num_val:]
y_train_OH = y_train_OH[num_val:]

# X_train = X_train.reshape([X_train.shape[0],-1])
# X_val = X_val.reshape([X_val.shape[0],-1])
# mean_image = np.mean(X_train, axis=0)
# X_train = X_train.astype(np.float32) - mean_image.astype(np.float32)
# X_val = X_val.astype(np.float32) - mean_image
# X_train = X_train.reshape([num_train-num_val,height,weight,channels])
# X_val = X_val.reshape([num_val,height,weight,channels])

print(X_train.shape,X_val.shape)

(2100, 224, 224, 3) (900, 224, 224, 3)


## InceptionV3

In [37]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer 
predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

### Training

In [38]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# save the best model
train_time = str(int(time.time()))
checkpointer = ModelCheckpoint(filepath='keras_model/inceptionV3_best_'+train_time+'.hdf5', verbose=1, save_best_only=True)

# train the model on the new data for a few epochs
model.fit(x=X_train, y=y_train_OH, batch_size=32,
          epochs=20, verbose=2, validation_data=(X_val, y_val_OH), shuffle = True, callbacks=[checkpointer])

Train on 2100 samples, validate on 900 samples
Epoch 1/20

Epoch 00001: val_loss improved from inf to 2.01083, saving model to keras_model/inceptionV3_best_1521494355.hdf5
 - 92s - loss: 1.9194 - val_loss: 2.0108
Epoch 2/20

Epoch 00002: val_loss improved from 2.01083 to 0.21936, saving model to keras_model/inceptionV3_best_1521494355.hdf5
 - 33s - loss: 0.3656 - val_loss: 0.2194
Epoch 3/20

Epoch 00003: val_loss improved from 0.21936 to 0.02147, saving model to keras_model/inceptionV3_best_1521494355.hdf5
 - 33s - loss: 0.2280 - val_loss: 0.0215
Epoch 4/20

Epoch 00004: val_loss did not improve
 - 34s - loss: 0.1563 - val_loss: 0.1690
Epoch 5/20

Epoch 00005: val_loss did not improve
 - 35s - loss: 0.1759 - val_loss: 0.0249
Epoch 6/20

Epoch 00006: val_loss did not improve
 - 34s - loss: 0.1951 - val_loss: 0.1110
Epoch 7/20

Epoch 00007: val_loss did not improve
 - 35s - loss: 0.1335 - val_loss: 0.0817
Epoch 8/20

Epoch 00008: val_loss did not improve
 - 34s - loss: 0.1343 - val_loss:

In [40]:
model.load_weights('keras_model/inceptionV3_best_1521494355.hdf5')

In [41]:
# val accuracy
np.mean(np.argmax(model.predict(X_val),axis=1)==np.argmax(y_val_OH, axis=1))

0.9922222222222222

In [42]:
# train accuracy
np.mean(np.argmax(model.predict(X_train),axis=1)==np.argmax(y_train_OH, axis=1))

0.9928571428571429

## Xception

In [1]:
from keras.applications.xception import Xception

E:\Program\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# create the base pre-trained model
base_model = Xception(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer
predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

83689472/83683744 [==============================] - 23s 0us/step


### Training

In [9]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# save the best model
train_time = str(int(time.time()))
checkpointer = ModelCheckpoint(filepath='keras_model/xception_best_'+train_time+'.hdf5', verbose=1, save_best_only=True)

# train the model on the new data for a few epochs
model.fit(x=X_train, y=y_train_OH, batch_size=32,
          epochs=20, verbose=2, validation_data=(X_val, y_val_OH), shuffle = True, callbacks=[checkpointer])

Train on 2100 samples, validate on 900 samples
Epoch 1/20

Epoch 00001: val_loss improved from inf to 0.04477, saving model to keras_model/xception_best_1521505472.hdf5
 - 56s - loss: 0.3402 - val_loss: 0.0448
Epoch 2/20

Epoch 00002: val_loss improved from 0.04477 to 0.02538, saving model to keras_model/xception_best_1521505472.hdf5
 - 47s - loss: 0.1076 - val_loss: 0.0254
Epoch 3/20

Epoch 00003: val_loss improved from 0.02538 to 0.01980, saving model to keras_model/xception_best_1521505472.hdf5
 - 48s - loss: 0.1308 - val_loss: 0.0198
Epoch 4/20

Epoch 00004: val_loss improved from 0.01980 to 0.01499, saving model to keras_model/xception_best_1521505472.hdf5
 - 49s - loss: 0.0667 - val_loss: 0.0150
Epoch 5/20

Epoch 00005: val_loss improved from 0.01499 to 0.01185, saving model to keras_model/xception_best_1521505472.hdf5
 - 49s - loss: 0.0650 - val_loss: 0.0118
Epoch 6/20

Epoch 00006: val_loss did not improve
 - 46s - loss: 0.0684 - val_loss: 0.1694
Epoch 7/20

Epoch 00007: val_lo

In [10]:
model.load_weights('keras_model/xception_best_1521505472.hdf5')

In [11]:
# val accuracy
np.mean(np.argmax(model.predict(X_val),axis=1)==np.argmax(y_val_OH, axis=1))

0.9944444444444445

In [12]:
# train accuracy
np.mean(np.argmax(model.predict(X_train),axis=1)==np.argmax(y_train_OH, axis=1))

0.9980952380952381

## Mobile Net

In [74]:
# create the base pre-trained model
base_model = MobileNet(input_shape=(224, 224, 3),weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer 
predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [75]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# save the best model
train_time = str(int(time.time()))
checkpointer = ModelCheckpoint(filepath='keras_model/mobilenet_best_'+train_time+'.hdf5', verbose=1, save_best_only=True)

# train the model on the new data for a few epochs
model.fit(x=X_train, y=y_train_OH, batch_size=32,
          epochs=30, verbose=2, validation_data=(X_val, y_val_OH), shuffle = True, callbacks=[checkpointer])

Train on 2100 samples, validate on 900 samples
Epoch 1/30

Epoch 00001: val_loss improved from inf to 0.04328, saving model to keras_model/mobilenet_best_1521594642.hdf5
 - 24s - loss: 1.8407 - val_loss: 0.0433
Epoch 2/30

Epoch 00002: val_loss did not improve
 - 10s - loss: 0.1837 - val_loss: 0.0578
Epoch 3/30

Epoch 00003: val_loss did not improve
 - 9s - loss: 0.1314 - val_loss: 0.0900
Epoch 4/30

Epoch 00004: val_loss did not improve
 - 10s - loss: 0.1106 - val_loss: 0.1551
Epoch 5/30

Epoch 00005: val_loss did not improve
 - 9s - loss: 0.1293 - val_loss: 1.1997
Epoch 6/30

Epoch 00006: val_loss did not improve
 - 9s - loss: 0.0871 - val_loss: 0.2188
Epoch 7/30

Epoch 00007: val_loss did not improve
 - 10s - loss: 0.0863 - val_loss: 0.4463
Epoch 8/30

Epoch 00008: val_loss did not improve
 - 9s - loss: 0.0723 - val_loss: 0.1097
Epoch 9/30

Epoch 00009: val_loss did not improve
 - 10s - loss: 0.0884 - val_loss: 0.3647
Epoch 10/30

Epoch 00010: val_loss did not improve
 - 10s - loss:

In [76]:
model.load_weights('keras_model/mobilenet_best_1521594642.hdf5')

In [77]:
# val accuracy
np.mean(np.argmax(model.predict(X_val),axis=1)==np.argmax(y_val_OH, axis=1))

0.9866666666666667

In [78]:
# train accuracy
np.mean(np.argmax(model.predict(X_train),axis=1)==np.argmax(y_train_OH, axis=1))

0.9923809523809524

## InceptionV3 with data augmentation

In [13]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer 
predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

## data generator

In [14]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range = 0.5,
    fill_mode = 'nearest',
)

datagen.fit(X_train)

In [15]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# save the best model
train_time = str(int(time.time()))
checkpointer = ModelCheckpoint(filepath='keras_model/InceptionV3_aug_'+train_time+'.hdf5', verbose=1, save_best_only=True)

# train the model on the new data for a few epochs
model.fit_generator(datagen.flow(X_train, y_train_OH, batch_size=32), validation_data=(X_val, y_val_OH),
                    steps_per_epoch=len(X_train) / 32, epochs=20,shuffle = True, callbacks=[checkpointer])

Epoch 1/20
65/65 [============================>.] - ETA: 0s - loss: 4.3534
Epoch 00001: val_loss improved from inf to 0.10769, saving model to keras_model/InceptionV3_aug_1521580658.hdf5
66/65 [==============================] - 62s 944ms/step - loss: 4.2931 - val_loss: 0.1077
Epoch 2/20
65/65 [============================>.] - ETA: 0s - loss: 0.3703
Epoch 00002: val_loss improved from 0.10769 to 0.03143, saving model to keras_model/InceptionV3_aug_1521580658.hdf5
66/65 [==============================] - 38s 582ms/step - loss: 0.3659 - val_loss: 0.0314
Epoch 3/20
65/65 [============================>.] - ETA: 0s - loss: 0.2622
Epoch 00003: val_loss did not improve
66/65 [==============================] - 38s 582ms/step - loss: 0.2606 - val_loss: 0.3852
Epoch 4/20
65/65 [============================>.] - ETA: 0s - loss: 0.2091
Epoch 00004: val_loss did not improve
66/65 [==============================] - 37s 562ms/step - loss: 0.2070 - val_loss: 0.0409
Epoch 5/20
65/65 [==================

In [22]:
model.load_weights('keras_model/InceptionV3_aug_1521580658.hdf5')

In [23]:
# val accuracy
np.mean(np.argmax(model.predict(X_val),axis=1)==np.argmax(y_val_OH, axis=1))

0.9911111111111112

In [24]:
# train accuracy
np.mean(np.argmax(model.predict(X_train),axis=1)==np.argmax(y_train_OH, axis=1))

0.9928571428571429

## Xception with data augmentation

In [29]:
# create the base pre-trained model
base_model = Xception(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer
predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [30]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range = 0.5,
    fill_mode = 'nearest',
)

datagen.fit(X_train)

In [31]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# save the best model
train_time = str(int(time.time()))
checkpointer = ModelCheckpoint(filepath='keras_model/Xception_aug_'+train_time+'.hdf5', verbose=1, save_best_only=True)

# train the model on the new data for a few epochs
model.fit_generator(datagen.flow(X_train, y_train_OH, batch_size=32), validation_data=(X_val, y_val_OH),
                    steps_per_epoch=len(X_train) / 32, epochs=20,shuffle = True, callbacks=[checkpointer])

Epoch 1/20
65/65 [============================>.] - ETA: 0s - loss: 0.5074
Epoch 00001: val_loss improved from inf to 0.08192, saving model to keras_model/Xception_aug_1521582759.hdf5
66/65 [==============================] - 100s 2s/step - loss: 0.4998 - val_loss: 0.0819
Epoch 2/20
65/65 [============================>.] - ETA: 0s - loss: 0.2243
Epoch 00002: val_loss improved from 0.08192 to 0.02438, saving model to keras_model/Xception_aug_1521582759.hdf5
66/65 [==============================] - 77s 1s/step - loss: 0.2213 - val_loss: 0.0244
Epoch 3/20
65/65 [============================>.] - ETA: 0s - loss: 0.1439
Epoch 00003: val_loss did not improve
66/65 [==============================] - 75s 1s/step - loss: 0.1421 - val_loss: 0.1145
Epoch 4/20
65/65 [============================>.] - ETA: 0s - loss: 0.1122
Epoch 00004: val_loss did not improve
66/65 [==============================] - 86s 1s/step - loss: 0.1108 - val_loss: 0.0372
Epoch 5/20
65/65 [============================>.] - E

In [32]:
model.load_weights('keras_model/Xception_aug_1521582759.hdf5')

In [33]:
# val accuracy
np.mean(np.argmax(model.predict(X_val),axis=1)==np.argmax(y_val_OH, axis=1))

0.9977777777777778

In [24]:
# train accuracy
np.mean(np.argmax(model.predict(X_train),axis=1)==np.argmax(y_train_OH, axis=1))

0.9928571428571429

### Prediction time on 3000 images (with GPU)

In [58]:
start = time.time()
model.predict(X_val)
model.predict(X_train)
time.time() - start

50.65471911430359

## Mobile Net with data augmentation

In [82]:
# create the base pre-trained model
base_model = MobileNet(input_shape=(224, 224, 3),weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer 
predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [83]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range = 0.5,
    fill_mode = 'nearest',
)

datagen.fit(X_train)

In [84]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# save the best model
train_time = str(int(time.time()))
checkpointer = ModelCheckpoint(filepath='keras_model/mobilenet_aug_'+train_time+'.hdf5', verbose=1, save_best_only=True)

# train the model on the new data for a few epochs
model.fit_generator(datagen.flow(X_train, y_train_OH, batch_size=32), validation_data=(X_val, y_val_OH),
                    steps_per_epoch=len(X_train) / 32, epochs=30,shuffle = True, callbacks=[checkpointer])

Epoch 1/30
65/65 [============================>.] - ETA: 0s - loss: 5.8901
Epoch 00001: val_loss improved from inf to 0.24856, saving model to keras_model/mobilenet_aug_1521595248.hdf5
66/65 [==============================] - 36s 543ms/step - loss: 5.8037 - val_loss: 0.2486
Epoch 2/30
65/65 [============================>.] - ETA: 0s - loss: 0.3031
Epoch 00002: val_loss improved from 0.24856 to 0.06660, saving model to keras_model/mobilenet_aug_1521595248.hdf5
66/65 [==============================] - 18s 277ms/step - loss: 0.3028 - val_loss: 0.0666
Epoch 3/30
65/65 [============================>.] - ETA: 0s - loss: 0.1850
Epoch 00003: val_loss improved from 0.06660 to 0.02315, saving model to keras_model/mobilenet_aug_1521595248.hdf5
66/65 [==============================] - 18s 273ms/step - loss: 0.1873 - val_loss: 0.0232
Epoch 4/30
65/65 [============================>.] - ETA: 0s - loss: 0.1466
Epoch 00004: val_loss did not improve
66/65 [==============================] - 18s 267ms/ste

In [85]:
model.load_weights('keras_model/mobilenet_aug_1521595248.hdf5')

In [86]:
# val accuracy
np.mean(np.argmax(model.predict(X_val),axis=1)==np.argmax(y_val_OH, axis=1))

0.9933333333333333

In [87]:
# train accuracy
np.mean(np.argmax(model.predict(X_train),axis=1)==np.argmax(y_train_OH, axis=1))

0.9976190476190476

In [88]:
start = time.time()
model.predict(X_val)
model.predict(X_train)
time.time() - start

10.559442520141602